In [ ]:
p!apt install ffmpeg
!pip install ray
!pip install ray[default]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pathlib
from time import time
from tqdm import tqdm
from glob import glob

import numpy as np
import IPython.display as ipd

# audio processing package
import librosa
import soundfile as sf

# multiprocessing package
os.environ['RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE'] = '1'
import ray

In [ ]:
print('librosa * ', librosa.__version__)

librosa *  0.8.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.getcwd()
os.chdir('/content/drive/MyDrive/ColabNotebooks/suyeon')
os.getcwd()

'/content/drive/MyDrive/ColabNotebooks/suyeon'

In [ ]:
def load_wav_librosa(wav_path):
    try: 
        wav, sr = librosa.load(wav_path, sr=22050)
        mono_wav = librosa.to_mono(wav)
        return mono_wav, sr  
    except Exception as e:
        return (wav_path, e)

def trim_normalize_librosa(wav, sr, save_path):
    try:
        wav = wav / max(abs(wav))  #오디오 정규화
        _, trim_index = librosa.effects.trim(wav, top_db=30)       # 3. 주파수, (오디오 앞 뒤 공백)무음 자르기 
        wav = wav[max(trim_index[0], 0):min(trim_index[1], len(wav))]
        sf.write(save_path, wav, sr, format="WAV", subtype='PCM_16')
    except Exception as e:
        return (save_path, e)

In [ ]:
@ray.remote
def preprocess_utterance(wavpath, outpath, re_sr = 22050):
    wav, sr = load_wav_librosa(wavpath)
    _result = trim_normalize_librosa(wav, sr, outpath)

In [ ]:
## define paths
_ddir = '.'
_odir = './librosa_wavs2'
pathlib.Path(_odir).mkdir(exist_ok=True)

wave_paths = glob(_ddir + '/wavs/*.wav')
out_paths = [os.path.join(_odir, wp.split('/')[-1]) for wp in wave_paths]

In [ ]:
wave_paths[:5]

['./wavs/0046e43d307932576c7c023134317315 (1).wav',
 './wavs/0002d3428f0ddfa5a48eec5cc351daa8 (1).wav',
 './wavs/0024cc0a9248edacfc48d16e5ffc945b (1).wav',
 './wavs/004b5ab65c6a7f0489af581c01897a31.wav',
 './wavs/0056e27aa382a043604eb86cf0b902c7.wav']

In [ ]:
## initialize ray
ray.init()

2022-05-27 02:38:14,506	INFO services.py:1462 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.7.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '172.28.0.2', 'raylet_ip_address': '172.28.0.2', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-27_02-38-12_341318_982/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-27_02-38-12_341318_982/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-27_02-38-12_341318_982', 'metrics_export_port': 58915, 'gcs_address': '172.28.0.2:57016', 'address': '172.28.0.2:57016', 'node_id': '72487d66412a2fce0cfc51dbd9d23439549a30f53fe62966507628bd'})

In [ ]:
t = time()
results = ray.get([preprocess_utterance.remote(wp, op) for wp, op in zip(wave_paths, out_paths)])
print(time() - t)

395.1921021938324


In [ ]:
## check result
[r for r in results if r is not None]

[]